<a href="https://colab.research.google.com/github/mcadete/LLM_4_Biz_16/blob/main/Tech16_HW5_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "crewai[tools]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:

In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

# Simple example

In [2]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# Define the research topic
topic = "Implications for Nvidia on the Deepseek model"

# Create the Researcher agent
researcher = Agent(
    name="Researcher",
    role="Research Analyst",
    goal="Gather reliable information on a topic",
    backstory="An experienced researcher skilled in extracting key insights from online sources.",
    tools=[SerperDevTool(), WebsiteSearchTool()],
    verbose=True
)

# Create the Writer agent
writer = Agent(
    name="Writer",
    role="Content Writer",
    goal="Summarize research findings clearly",
    backstory="A skilled writer who crafts well-structured summaries from research data.",
    verbose=True
)

# Define tasks
research_task = Task(
    description=f"Research '{topic}' and summarize key findings.",
    expected_output=f"Bullet points summarizing '{topic}'.",
    agent=researcher
)

write_task = Task(
    description=f"Write a structured report on '{topic}' using research findings.",
    expected_output=f"A markdown summary of '{topic}' with clear sections.",
    agent=writer,
    context=[research_task]
)

# Assemble the crew and run tasks
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential
)

result = crew.kickoff()

# Display the final report
display(Markdown(result.raw))


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

# Agent: Research Analyst
## Task: Research 'Implications for Nvidia on the Deepseek model' and summarize key findings.


# Agent: Research Analyst
## Thought: I need to research the implications for Nvidia regarding the Deepseek model, gathering reliable and relevant information on this topic.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Implications for Nvidia on the Deepseek model\"}"
## Tool Output: 
{'searchParameters': {'q': 'Implications for Nvidia on the Deepseek model', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "How DeepSeek's AI Model Impacts Nvidia Stock - Nasdaq", 'link': 'https://www.nasdaq.com/articles/how-deepseeks-ai-model-impacts-nvidia-stock', 'snippet': 'We believe this development could potentially have implications for Nvidia stock (NASDAQ:NVDA), the dominant player in the AI hardware space.', 'position': 1}, {'title': 'Nvidia, AI Chip Peers Tumble On DeepSeek Implications', 'link': 'https://www.

# Implications for Nvidia on the Deepseek Model

## Introduction
The recent launch of Deepseek's AI model has significant implications for Nvidia, highlighting both challenges and opportunities in the rapidly evolving AI hardware landscape. This report delves into the potential effects on Nvidia's market position, pricing strategy, and overall competitiveness in light of Deepseek's innovations.

## Market Reaction
Following the announcement of the Deepseek model, Nvidia experienced a notable decline in its stock price. This shift reflects investor apprehension regarding intensified competition in the AI hardware space. Investors are concerned that the emergence of Deepseek could undermine Nvidia's dominance, prompting a reassessment of future profitability.

## Potential Price War
Reports suggest that Deepseek's model was developed at a substantially lower cost than existing alternatives. This cost efficiency raises concerns about the possibility of a price war within the AI technology market. Should Deepseek aggressively price its offerings, Nvidia may be pressured to adjust its pricing strategy, which could lead to reduced profit margins and lost market share.

## Impact on Hardware Demand
As Deepseek introduces cost-effective and high-performance alternatives, Nvidia faces potential declines in the demand for its GPUs, which have historically been a core revenue driver. The shift towards lower-cost solutions indicates that budget-conscious consumers and businesses may gravitate toward Deepseek's offerings, challenging Nvidia's sales volumes.

## Response from Nvidia's CEO
In light of the competitive threat posed by Deepseek, Nvidia's CEO, Jensen Huang, remains optimistic. He posits that innovations from new entrants like Deepseek could stimulate broader demand for AI hardware, suggesting that increased competition may ultimately benefit Nvidia's market position. Huang's confidence implies that Nvidia may leverage its established brand and expertise to sustain its leadership amidst rising competition.

## Investment in Technology
The effective incorporation of Nvidia chips within Deepseek's AI model highlights a critical aspect of this competitive dynamic. While Deepseek's model can attract consumers with its budget-friendly solutions, it raises concerns about the return on investment for Nvidia's high-end chips. If Deepseek's model continues to capture market share, Nvidia may need to re-evaluate its product offerings to ensure they align with changing market demands and consumer preferences.

## Competitive Landscape
The introduction of Deepseek’s AI model has indeed reshaped the competitive landscape for AI hardware. With additional players entering the market, Nvidia is compelled to accelerate its innovation efforts. To maintain its industry leadership, Nvidia must enhance the capabilities of its products and develop strategies that differentiate its offerings from those of competitors.

## Conclusion
In summary, the implications of Deepseek's AI model for Nvidia are multifaceted, ranging from stock market reactions to pressures on pricing strategies and hardware demand. While challenges exist, Nvidia's proactive stance and commitment to innovation will be crucial in navigating this changing landscape. Continued vigilance and adaptation will enable Nvidia to thrive despite the emerging competition, reinforcing its position as a leader in AI hardware.

# More detailed

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# --------------------------
# Configuration for a generic research task
# --------------------------
# Set your research topic and task-specific descriptions.
research_topic = "What are the implications for Nvidia on the Deepseek model?"
research_description = (
    f"Research the topic '{research_topic}'. Provide a summary of key findings with supporting details."
)
research_expected_output = (
    f"A collection of bullet points detailing the major insights about '{research_topic}', including relevant sources."
)

writing_description = (
    f"Compose an analysis report summarizing the findings on '{research_topic}'. "
    "The report should be structured, easy to read, and include clear headings and bullet points."
)
writing_expected_output = (
    f"A well-structured markdown summary of research findings on '{research_topic}', with clear headings and references."
)

# --------------------------
# Set up the tools for the Researcher agent: a web search tool and a web content reader.
# --------------------------
search_tool = SerperDevTool()        # Enables web searching via the Serper API
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages

# --------------------------
# Create the Researcher agent, whose role is to gather information from the web.
# --------------------------
researcher_agent = Agent(
    name="Researcher",
    role="Research Analyst",
    goal="Gather reliable information on any given topic",
    backstory="An expert researcher adept at finding and curating information from diverse sources on the web.",
    tools=[search_tool, web_reader_tool],
    verbose=True  # set to True to see internal processing details
)

# --------------------------
# Create the Writer agent, whose role is to compile a summary report based on research.
# --------------------------
writer_agent = Agent(
    name="Writer",
    role="Content Writer",
    goal="Summarize research findings into a clear report",
    backstory="A skilled writer who can explain complex information in simple terms.",
    tools=[],  # No special tools needed for this summarization task
    verbose=True
)

# --------------------------
# Define the research task for the Researcher agent.
# --------------------------
research_task = Task(
    description=research_description,
    expected_output=research_expected_output,
    agent=researcher_agent
)

# --------------------------
# Define the writing/summarization task for the Writer agent.
# --------------------------
write_task = Task(
    description=writing_description,
    expected_output=writing_expected_output,
    agent=writer_agent,
    context=[research_task]  # This task uses the output of the research_task as context
)

# --------------------------
# Assemble the crew with both agents and tasks.
# --------------------------
research_crew = Crew(
    agents=[researcher_agent, writer_agent],
    tasks=[research_task, write_task],
    process=Process.sequential,  # Runs the research task before the writing task
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final analysis report as markdown
display(Markdown(result.raw))

### Include critic

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# --------------------------
# Configuration for a generic research task
# --------------------------
research_topic = "What are the implications for Nvidia on the Deepseek model?"
research_description = (
    f"Research the topic '{research_topic}'. Provide a summary of key findings with supporting details."
)
research_expected_output = (
    f"A collection of bullet points detailing the major insights about '{research_topic}', including relevant sources."
)

writing_description = (
    f"Compose an analysis report summarizing the findings on '{research_topic}'. "
    "The report should be structured, easy to read, and include clear headings and bullet points."
)
writing_expected_output = (
    f"A well-structured markdown summary of research findings on '{research_topic}', with clear headings and references."
)

critic_description = (
    f"Review the analysis report summarizing the findings on '{research_topic}'. "
    "Provide detailed feedback on the clarity, coherence, and overall quality of the report, "
    "including strengths, weaknesses, and suggestions for improvement."
)
critic_expected_output = (
    "A collection of bullet points or paragraphs providing constructive critique of the analysis report, "
    "highlighting strengths, weaknesses, and potential improvements."
)

rewriter_description = (
    f"Rewrite the analysis report on '{research_topic}' by incorporating the feedback from the Critic. "
    "Produce a refined version that addresses the criticisms and improves clarity and structure."
)
rewriter_expected_output = (
    f"A refined markdown analysis report on '{research_topic}', incorporating the critic's feedback, "
    "with clear headings and detailed explanations."
)

# --------------------------
# Set up the tools for the Researcher agent: a web search tool and a web content reader.
# --------------------------
search_tool = SerperDevTool()         # Enables web searching via the Serper API
web_reader_tool = WebsiteSearchTool()   # Retrieves and reads content from web pages

# --------------------------
# Create the Researcher agent
# --------------------------
researcher_agent = Agent(
    name="Researcher",
    role="Research Analyst",
    goal="Gather reliable information on any given topic",
    backstory="An expert researcher adept at finding and curating information from diverse sources on the web.",
    tools=[search_tool, web_reader_tool],
    verbose=True
)

# --------------------------
# Create the Writer agent
# --------------------------
writer_agent = Agent(
    name="Writer",
    role="Content Writer",
    goal="Summarize research findings into a clear report",
    backstory="A skilled writer who can explain complex information in simple terms.",
    tools=[],  # No special tools needed for this summarization task
    verbose=True
)

# --------------------------
# Create the Critic agent
# --------------------------
critic_agent = Agent(
    name="Critic",
    role="Content Critic",
    goal="Review and provide constructive feedback on the analysis report",
    backstory="An expert in critically evaluating content to ensure quality, accuracy, and clarity.",
    tools=[],  # No special tools needed for this critique task
    verbose=True
)

# --------------------------
# Create the Rewriter agent
# --------------------------
rewriter_agent = Agent(
    name="Rewriter",
    role="Content Editor",
    goal="Rewrite the analysis report by incorporating the critic's feedback",
    backstory="A skilled editor who refines content to enhance clarity, coherence, and overall quality.",
    tools=[],  # No special tools needed for this rewriting task
    verbose=True
)

# --------------------------
# Define the research task for the Researcher agent.
# --------------------------
research_task = Task(
    description=research_description,
    expected_output=research_expected_output,
    agent=researcher_agent
)

# --------------------------
# Define the writing/summarization task for the Writer agent.
# --------------------------
write_task = Task(
    description=writing_description,
    expected_output=writing_expected_output,
    agent=writer_agent,
    context=[research_task]  # This task uses the output of the research_task as context
)

# --------------------------
# Define the critique task for the Critic agent.
# --------------------------
critic_task = Task(
    description=critic_description,
    expected_output=critic_expected_output,
    agent=critic_agent,
    context=[write_task]  # This task uses the output of the write_task as context
)

# --------------------------
# Define the rewriting task for the Rewriter agent.
# --------------------------
rewriter_task = Task(
    description=rewriter_description,
    expected_output=rewriter_expected_output,
    agent=rewriter_agent,
    context=[write_task, critic_task]  # This task uses both the original report and the critique as context
)

# --------------------------
# Assemble the crew with all four agents and tasks.
# --------------------------
research_crew = Crew(
    agents=[researcher_agent, writer_agent, critic_agent, rewriter_agent],
    tasks=[research_task, write_task, critic_task, rewriter_task],
    process=Process.sequential,  # Executes tasks in sequence: research -> writing -> critique -> rewrite
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final refined report as markdown
display(Markdown(result.raw))


# Hierarchical

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# Import ChatOpenAI to serve as the manager LLM for the hierarchical process
from langchain.chat_models import ChatOpenAI

# --------------------------
# Configuration for a generic research task
# --------------------------
research_topic = "What are the implications for Nvidia on the Deepseek model?"
research_description = (
    f"Research the topic '{research_topic}'. Provide a summary of key findings with supporting details."
)
research_expected_output = (
    f"A collection of bullet points detailing the major insights about '{research_topic}', including relevant sources."
)

writing_description = (
    f"Compose an analysis report summarizing the findings on '{research_topic}'. "
    "The report should be structured, easy to read, and include clear headings and bullet points."
)
writing_expected_output = (
    f"A well-structured markdown summary of research findings on '{research_topic}', with clear headings and references."
)

# --------------------------
# Set up the tools for the Researcher agent: a web search tool and a web content reader.
# --------------------------
search_tool = SerperDevTool()        # Enables web searching via the Serper API
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages

# --------------------------
# Create the Researcher agent, whose role is to gather information from the web.
# --------------------------
researcher_agent = Agent(
    name="Researcher",
    role="Research Analyst",
    goal="Gather reliable information on any given topic",
    backstory="An expert researcher adept at finding and curating information from diverse sources on the web.",
    tools=[search_tool, web_reader_tool],
    verbose=True
)

# --------------------------
# Create the Writer agent, whose role is to compile a summary report based on research.
# --------------------------
writer_agent = Agent(
    name="Writer",
    role="Content Writer",
    goal="Summarize research findings into a clear report",
    backstory="A skilled writer who can explain complex information in simple terms.",
    tools=[],
    verbose=True
)

# --------------------------
# Define the research task for the Researcher agent.
# --------------------------
research_task = Task(
    description=research_description,
    expected_output=research_expected_output,
    agent=researcher_agent
)

# --------------------------
# Define the writing/summarization task for the Writer agent.
# --------------------------
write_task = Task(
    description=writing_description,
    expected_output=writing_expected_output,
    agent=writer_agent,
    context=[research_task]  # This task uses the output of the research_task as context
)

# --------------------------
# Set up the LLM-based manager for the hierarchical process.
# --------------------------
manager_llm = ChatOpenAI(temperature=0, model="gpt-4")

# --------------------------
# Assemble the crew with both agents and tasks using a hierarchical process.
# --------------------------
research_crew = Crew(
    agents=[researcher_agent, writer_agent],
    tasks=[research_task, write_task],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=manager_llm,         # Provide the LLM-based manager required for hierarchical execution
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final analysis report as markdown
display(Markdown(result.raw))


### Include critic

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# Import ChatOpenAI to serve as the manager LLM for the hierarchical process
from langchain.chat_models import ChatOpenAI

# --------------------------
# Configuration for a research task with critic feedback and revision
# --------------------------
research_topic = "What are the implications for Nvidia on the Deepseek model?"

# Task descriptions and expected outputs
research_description = (
    f"Research the topic '{research_topic}'. Provide a summary of key findings with supporting details."
)
research_expected_output = (
    f"A collection of bullet points detailing the major insights about '{research_topic}', including relevant sources."
)

writing_description = (
    f"Compose an initial analysis report summarizing the findings on '{research_topic}'. "
    "The report should be structured, easy to read, and include clear headings and bullet points."
)
writing_expected_output = (
    f"A well-structured markdown summary of research findings on '{research_topic}', with clear headings and references."
)

critique_description = (
    f"Review the initial report on '{research_topic}' and provide detailed feedback focusing on structure, clarity, and insights. "
    "Offer suggestions for improvement and highlight any gaps or areas needing clarification."
)
critique_expected_output = (
    f"A set of bullet points outlining the strengths, weaknesses, and actionable recommendations for the report on '{research_topic}'."
)

rewrite_description = (
    f"Rewrite the report on '{research_topic}' incorporating the feedback provided. "
    "Ensure the revised report is improved in clarity, structure, and depth of analysis."
)
rewrite_expected_output = (
    f"An updated markdown summary of research findings on '{research_topic}', refined based on the critique feedback."
)

# --------------------------
# Set up the tools for the Researcher agent: a web search tool and a web content reader.
# --------------------------
search_tool = SerperDevTool()        # Enables web searching via the Serper API
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages

# --------------------------
# Create the Researcher agent, whose role is to gather information from the web.
# --------------------------
researcher_agent = Agent(
    name="Researcher",
    role="Research Analyst",
    goal="Gather reliable information on any given topic",
    backstory="An expert researcher adept at finding and curating information from diverse sources on the web.",
    tools=[search_tool, web_reader_tool],
    verbose=True
)

# --------------------------
# Create the Writer agent, whose role is to compile a summary report based on research.
# --------------------------
writer_agent = Agent(
    name="Writer",
    role="Content Writer",
    goal="Summarize research findings into a clear report",
    backstory="A skilled writer who can explain complex information in simple terms.",
    tools=[],
    verbose=True
)

# --------------------------
# Create the Critic agent, whose role is to provide detailed feedback on the report.
# --------------------------
critic_agent = Agent(
    name="Critic",
    role="Content Critic",
    goal="Evaluate and provide constructive feedback on reports",
    backstory="An experienced critic with a keen eye for detail, focused on enhancing clarity, structure, and depth in reports.",
    tools=[],
    verbose=True
)

# --------------------------
# Define the research task for the Researcher agent.
# --------------------------
research_task = Task(
    description=research_description,
    expected_output=research_expected_output,
    agent=researcher_agent
)

# --------------------------
# Define the initial writing task for the Writer agent.
# --------------------------
initial_write_task = Task(
    description=writing_description,
    expected_output=writing_expected_output,
    agent=writer_agent,
    context=[research_task]  # Uses the output of the research task as context
)

# --------------------------
# Define the critique task for the Critic agent.
# --------------------------
critique_task = Task(
    description=critique_description,
    expected_output=critique_expected_output,
    agent=critic_agent,
    context=[initial_write_task]  # Uses the initial report as context
)

# --------------------------
# Define the rewrite task for the Writer agent to update the report based on the critique.
# --------------------------
rewrite_task = Task(
    description=rewrite_description,
    expected_output=rewrite_expected_output,
    agent=writer_agent,
    context=[research_task, initial_write_task, critique_task]  # Uses both research and critique feedback as context
)

# --------------------------
# Set up the LLM-based manager for the hierarchical process.
# --------------------------
manager_llm = ChatOpenAI(temperature=0, model="gpt-4")

# --------------------------
# Assemble the crew with all agents and tasks using a hierarchical process.
# --------------------------
research_crew = Crew(
    agents=[researcher_agent, writer_agent, critic_agent],
    tasks=[research_task, initial_write_task, critique_task, rewrite_task],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=manager_llm,         # Provide the LLM-based manager required for hierarchical execution
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final updated analysis report as markdown
display(Markdown(result.raw))
